# Comprehensive Kidney Risk Factor Prediction Analysis

**Created:** 2025-11-20

**Dataset:** synthetic_kidney_risk.csv

## Overview

This notebook provides a comprehensive analysis of kidney disease risk factors using machine learning techniques. We analyze patient data including demographic information, laboratory results, and derived risk scores to predict kidney disease risk categories.

### Key Objectives:
1. Verify formula calculations for risk scoring
2. Perform exploratory data analysis
3. Train and compare multiple classification models
4. Analyze feature importance for risk prediction

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Utilities
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Metrics
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    precision_score, recall_score, f1_score
)
from sklearn.inspection import permutation_importance

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print('Libraries imported successfully!')

In [ ]:
# Load the dataset
data = pd.read_csv('synthetic_kidney_risk.csv')

print('Dataset loaded successfully!')
print(f'\nDataset Shape: {data.shape}')
print(f'Rows: {data.shape[0]}, Columns: {data.shape[1]}')

In [ ]:
# Display dataset information
print('Dataset Info:')
print('=' * 80)
data.info()

In [ ]:
# Display statistical summary
print('Statistical Summary:')
print('=' * 80)
data.describe()

In [ ]:
# Display first few rows
print('First 10 rows of the dataset:')
print('=' * 80)
data.head(10)

## Formula Verification

We verify the following formulas used in the dataset:

1. **Composite Score Formula:** `Composite_Score = 0.7 * S_ACR + 0.3 * S_ML`
2. **Risk Category Thresholds:**
   - Low: Composite_Score < 35
   - Moderate: 35 ≤ Composite_Score < 65
   - High: Composite_Score ≥ 65

In [ ]:
# Verify Composite Score formula
data['Composite_Score_Calculated'] = 0.7 * data['S_ACR'] + 0.3 * data['S_ML']

# Check if calculated scores match the dataset scores
score_difference = abs(data['Composite_Score'] - data['Composite_Score_Calculated'])
print('Composite Score Formula Verification:')
print('=' * 80)
print(f'Maximum difference: {score_difference.max():.6f}')
print(f'Mean difference: {score_difference.mean():.6f}')
print(f'All scores match (tolerance 0.01): {(score_difference < 0.01).all()}')

# Display sample calculations
print('\nSample Calculations:')
print(data[['S_ACR', 'S_ML', 'Composite_Score', 'Composite_Score_Calculated']].head(10))

# Drop the calculated column
data.drop('Composite_Score_Calculated', axis=1, inplace=True)

In [ ]:
# Verify Risk Category thresholds
def verify_risk_category(row):
    score = row['Composite_Score']
    if score < 35:
        return 'Low'
    elif score < 65:
        return 'Moderate'
    else:
        return 'High'

data['Risk_Category_Calculated'] = data.apply(verify_risk_category, axis=1)

# Check if categories match
categories_match = (data['Risk_Category'] == data['Risk_Category_Calculated']).all()

print('Risk Category Threshold Verification:')
print('=' * 80)
print(f'All categories match: {categories_match}')

# Display threshold analysis
print('\nThreshold Analysis:')
for category in ['Low', 'Moderate', 'High']:
    cat_data = data[data['Risk_Category'] == category]['Composite_Score']
    print(f'\n{category}:')
    print(f'  Count: {len(cat_data)}')
    print(f'  Min: {cat_data.min():.2f}')
    print(f'  Max: {cat_data.max():.2f}')
    print(f'  Mean: {cat_data.mean():.2f}')

# Drop the calculated column
data.drop('Risk_Category_Calculated', axis=1, inplace=True)

## Exploratory Data Analysis (EDA)

We perform comprehensive exploratory analysis to understand the distribution of features and their relationships.

In [ ]:
# Feature distributions
numerical_features = ['Age', 'Urine_Creatinine_mg_dL', 'Urine_Albumin_mg_L', 
                     'ACR_mg_per_g', 'pH', 'Specific_Gravity', 'Serum_Creatinine_mg_dL']

fig, axes = plt.subplots(4, 2, figsize=(15, 16))
axes = axes.ravel()

for idx, feature in enumerate(numerical_features):
    axes[idx].hist(data[feature], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(feature, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].set_title(f'Distribution of {feature}', fontsize=11, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

# Hide the last subplot if we have an odd number of features
if len(numerical_features) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()

print('Feature distributions plotted successfully!')

In [ ]:
# Correlation heatmap
plt.figure(figsize=(14, 10))

# Select only numeric columns for correlation
numeric_data = data.select_dtypes(include=[np.number])
correlation = numeric_data.corr()

sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of Numerical Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print('Correlation heatmap created successfully!')

In [ ]:
# Box plots by Risk Category
fig, axes = plt.subplots(4, 2, figsize=(15, 16))
axes = axes.ravel()

for idx, feature in enumerate(numerical_features):
    data.boxplot(column=feature, by='Risk_Category', ax=axes[idx])
    axes[idx].set_xlabel('Risk Category', fontsize=10)
    axes[idx].set_ylabel(feature, fontsize=10)
    axes[idx].set_title(f'{feature} by Risk Category', fontsize=11, fontweight='bold')
    axes[idx].get_figure().suptitle('')  # Remove the automatic title

# Hide the last subplot if we have an odd number of features
if len(numerical_features) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()

print('Box plots created successfully!')

## Data Preprocessing

We prepare the data for machine learning by:
1. Selecting relevant features
2. Encoding categorical variables
3. Splitting into train/test sets (80/20)
4. Scaling features using StandardScaler

In [ ]:
# Encode Sex (M=1, F=0)
data_processed = data.copy()
data_processed['Sex'] = data_processed['Sex'].map({'M': 1, 'F': 0})

# Define features and target
feature_cols = ['Age', 'Sex', 'Hypertension', 'Diabetes', 
                'Urine_Creatinine_mg_dL', 'Urine_Albumin_mg_L', 
                'ACR_mg_per_g', 'pH', 'Specific_Gravity', 'Serum_Creatinine_mg_dL']

X = data_processed[feature_cols]
y = data_processed['Risk_Category']

print('Features and target defined:')
print(f'Feature matrix shape: {X.shape}')
print(f'Target vector shape: {y.shape}')
print(f'\nFeatures: {feature_cols}')
print(f'\nTarget distribution:')
print(y.value_counts())

In [ ]:
# Split data into train and test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('Data split into train and test sets:')
print(f'Training set size: {X_train.shape[0]} samples')
print(f'Test set size: {X_test.shape[0]} samples')
print(f'\nTraining set target distribution:')
print(y_train.value_counts())
print(f'\nTest set target distribution:')
print(y_test.value_counts())

In [ ]:
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Features scaled using StandardScaler')
print(f'\nScaled training set shape: {X_train_scaled.shape}')
print(f'Scaled test set shape: {X_test_scaled.shape}')
print(f'\nSample of scaled features (first 5 samples, first 5 features):')
print(X_train_scaled[:5, :5])

## Model Training

We train four different classification models with hyperparameter tuning using GridSearchCV:
1. Random Forest Classifier
2. Gradient Boosting Classifier
3. Support Vector Machine (SVM)
4. Logistic Regression

In [ ]:
# Random Forest with GridSearchCV
print('Training Random Forest Classifier...')
print('=' * 80)

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train_scaled, y_train)

print(f'\nBest parameters: {rf_model.best_params_}')
print(f'Best cross-validation score: {rf_model.best_score_:.4f}')

# Predictions
rf_train_pred = rf_model.predict(X_train_scaled)
rf_test_pred = rf_model.predict(X_test_scaled)

# Evaluate
rf_train_acc = accuracy_score(y_train, rf_train_pred)
rf_test_acc = accuracy_score(y_test, rf_test_pred)

print(f'\nTraining accuracy: {rf_train_acc:.4f}')
print(f'Test accuracy: {rf_test_acc:.4f}')
print('\nClassification Report (Test Set):')
print(classification_report(y_test, rf_test_pred))

In [ ]:
# Gradient Boosting with GridSearchCV
print('Training Gradient Boosting Classifier...')
print('=' * 80)

gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

gb_model = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid=gb_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

gb_model.fit(X_train_scaled, y_train)

print(f'\nBest parameters: {gb_model.best_params_}')
print(f'Best cross-validation score: {gb_model.best_score_:.4f}')

# Predictions
gb_train_pred = gb_model.predict(X_train_scaled)
gb_test_pred = gb_model.predict(X_test_scaled)

# Evaluate
gb_train_acc = accuracy_score(y_train, gb_train_pred)
gb_test_acc = accuracy_score(y_test, gb_test_pred)

print(f'\nTraining accuracy: {gb_train_acc:.4f}')
print(f'Test accuracy: {gb_test_acc:.4f}')
print('\nClassification Report (Test Set):')
print(classification_report(y_test, gb_test_pred))

In [ ]:
# SVM with GridSearchCV
print('Training Support Vector Machine...')
print('=' * 80)

svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto', 0.001, 0.01]
}

svm_model = GridSearchCV(
    SVC(random_state=42),
    param_grid=svm_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

svm_model.fit(X_train_scaled, y_train)

print(f'\nBest parameters: {svm_model.best_params_}')
print(f'Best cross-validation score: {svm_model.best_score_:.4f}')

# Predictions
svm_train_pred = svm_model.predict(X_train_scaled)
svm_test_pred = svm_model.predict(X_test_scaled)

# Evaluate
svm_train_acc = accuracy_score(y_train, svm_train_pred)
svm_test_acc = accuracy_score(y_test, svm_test_pred)

print(f'\nTraining accuracy: {svm_train_acc:.4f}')
print(f'Test accuracy: {svm_test_acc:.4f}')
print('\nClassification Report (Test Set):')
print(classification_report(y_test, svm_test_pred))

In [ ]:
# Logistic Regression with GridSearchCV
print('Training Logistic Regression...')
print('=' * 80)

lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [200, 500, 1000]
}

lr_model = GridSearchCV(
    LogisticRegression(random_state=42),
    param_grid=lr_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

lr_model.fit(X_train_scaled, y_train)

print(f'\nBest parameters: {lr_model.best_params_}')
print(f'Best cross-validation score: {lr_model.best_score_:.4f}')

# Predictions
lr_train_pred = lr_model.predict(X_train_scaled)
lr_test_pred = lr_model.predict(X_test_scaled)

# Evaluate
lr_train_acc = accuracy_score(y_train, lr_train_pred)
lr_test_acc = accuracy_score(y_test, lr_test_pred)

print(f'\nTraining accuracy: {lr_train_acc:.4f}')
print(f'Test accuracy: {lr_test_acc:.4f}')
print('\nClassification Report (Test Set):')
print(classification_report(y_test, lr_test_pred))

## Model Comparison

We compare all four models based on accuracy, precision, recall, and F1-score.

In [ ]:
# Calculate metrics for all models
models = {
    'Random Forest': rf_test_pred,
    'Gradient Boosting': gb_test_pred,
    'SVM': svm_test_pred,
    'Logistic Regression': lr_test_pred
}

results = []
for model_name, predictions in models.items():
    results.append({
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, predictions),
        'Precision': precision_score(y_test, predictions, average='weighted'),
        'Recall': recall_score(y_test, predictions, average='weighted'),
        'F1-Score': f1_score(y_test, predictions, average='weighted')
    })

results_df = pd.DataFrame(results)
results_df = results_df.round(4)

print('Model Comparison Results:')
print('=' * 80)
print(results_df.to_string(index=False))

# Find best model
best_model_idx = results_df['Accuracy'].idxmax()
best_model = results_df.loc[best_model_idx, 'Model']
best_accuracy = results_df.loc[best_model_idx, 'Accuracy']

print(f'\nBest Model: {best_model} (Accuracy: {best_accuracy:.4f})')

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12']

for idx, (ax, metric) in enumerate(zip(axes.flat, metrics)):
    ax.bar(results_df['Model'], results_df[metric], color=colors[idx], alpha=0.7, edgecolor='black')
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim([0, 1.1])
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(results_df[metric]):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print('Model comparison visualization created successfully!')

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, (model_name, predictions) in enumerate(models.items()):
    cm = confusion_matrix(y_test, predictions, labels=['Low', 'Moderate', 'High'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Low', 'Moderate', 'High'],
                yticklabels=['Low', 'Moderate', 'High'],
                cbar_kws={'label': 'Count'})
    axes[idx].set_xlabel('Predicted', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=11, fontweight='bold')
    axes[idx].set_title(f'Confusion Matrix - {model_name}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print('Confusion matrices created successfully!')

## Feature Importance Analysis

We analyze feature importance using:
1. Random Forest built-in feature importance
2. Permutation importance on the best model

In [ ]:
# Feature importance from Random Forest
rf_best_model = rf_model.best_estimator_
feature_importance = rf_best_model.feature_importances_

# Create DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print('Feature Importance (Random Forest):')
print('=' * 80)
print(importance_df.to_string(index=False))

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], 
         color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.ylabel('Feature', fontsize=12, fontweight='bold')
plt.title('Feature Importance - Random Forest', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print('\nFeature importance visualization created successfully!')

In [ ]:
# Permutation importance
print('Calculating Permutation Importance...')
print('=' * 80)

perm_importance = permutation_importance(
    rf_best_model, X_test_scaled, y_test,
    n_repeats=10, random_state=42, n_jobs=-1
)

# Create DataFrame for permutation importance
perm_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance_Mean': perm_importance.importances_mean,
    'Importance_Std': perm_importance.importances_std
}).sort_values('Importance_Mean', ascending=False)

print('\nPermutation Importance:')
print(perm_importance_df.to_string(index=False))

# Visualize permutation importance
plt.figure(figsize=(12, 8))
plt.barh(perm_importance_df['Feature'], perm_importance_df['Importance_Mean'],
         xerr=perm_importance_df['Importance_Std'],
         color='coral', edgecolor='black', alpha=0.7)
plt.xlabel('Permutation Importance', fontsize=12, fontweight='bold')
plt.ylabel('Feature', fontsize=12, fontweight='bold')
plt.title('Permutation Importance - Random Forest', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print('\nPermutation importance visualization created successfully!')

In [ ]:
# Compare both importance methods
comparison_df = pd.DataFrame({
    'Feature': feature_cols,
    'RF_Importance': feature_importance,
    'Permutation_Importance': perm_importance.importances_mean
}).sort_values('RF_Importance', ascending=False)

fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(feature_cols))
width = 0.35

ax.bar(x - width/2, comparison_df['RF_Importance'], width, 
       label='RF Feature Importance', color='steelblue', edgecolor='black', alpha=0.7)
ax.bar(x + width/2, comparison_df['Permutation_Importance'], width,
       label='Permutation Importance', color='coral', edgecolor='black', alpha=0.7)

ax.set_xlabel('Features', fontsize=12, fontweight='bold')
ax.set_ylabel('Importance', fontsize=12, fontweight='bold')
ax.set_title('Comparison of Feature Importance Methods', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Feature'], rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('Feature importance comparison visualization created successfully!')

## Summary and Conclusions

This comprehensive analysis has:

1. ✅ Verified the Composite Score formula: `Composite_Score = 0.7 * S_ACR + 0.3 * S_ML`
2. ✅ Verified Risk Category thresholds: Low (<35), Moderate (35-65), High (≥65)
3. ✅ Performed extensive exploratory data analysis
4. ✅ Trained and compared 4 different models:
   - Random Forest
   - Gradient Boosting
   - Support Vector Machine
   - Logistic Regression
5. ✅ Analyzed feature importance using multiple methods

### Key Findings:

- All models performed well with proper hyperparameter tuning
- Feature importance analysis reveals which factors contribute most to kidney disease risk
- The analysis provides a solid foundation for risk prediction and clinical decision support

### Next Steps:

- Consider ensemble methods combining multiple models
- Explore additional feature engineering
- Validate on external datasets
- Deploy the best model for clinical use